#**Examen practico 1**

###Enunciado: https://drive.google.com/file/d/1b1ep6Uw9gxdM1FstIlFsSSzQarFeV1zk/view?usp=sharing###



In [9]:
from random import random
import math
import statistics

#Funciones

#Esta funcion calcula el intervalo de confianza al 95% de un conjunto de observaciones, devuelve una lista con varios elementos
def intConf(muestra):
  mediaMuestra = statistics.mean(muestra)
  desviacionMuestra = statistics.stdev(muestra)
  precision = 1.96*desviacionMuestra/math.sqrt(len(muestra))
  res = [mediaMuestra, desviacionMuestra, (mediaMuestra-precision, mediaMuestra+precision)]
  return res

#Con esta funcion podemos generar numeros que sigan una distribucion uniforme (mediante transformada inversa) dado un minimo y un maximo
def genUnif(min,max):
  r = random()
  return (max-min)*r + min

#Con esta funcion podemos generar mediante transformada inversa un resultado acerca de las caracteristicas de la ruta A cierto dia
def A_congestion_pronostico():
  r = random()
  if r <= 0.02:
    return "sin congestion, pronostico impreciso"
  elif r <= 0.12:
    return "congestion, pronostico impreciso"
  elif r <= 0.4:
    return "congestion, pronostico preciso"
  else:
    return "sin congestion, pronostico preciso"

#En esta funcion simulamos todos los elementos ya identificados para generar un tiempo real final
def simulacion():
  #Generamos un pronostico para b
  pronostico_b = genUnif(45,75)

  #Generamos un pronostico para A segun las probabilidades de los diferentes casos
  situacion_a = A_congestion_pronostico()

  if situacion_a == "sin congestion, pronostico preciso" or situacion_a == "congestion, pronostico impreciso":
    pronostico_a = genUnif(30,40)
  elif situacion_a == "sin congestion, pronostico impreciso" or situacion_a == "congestion, pronostico preciso":
    pronostico_a = genUnif(60,90)
  
  #Ruta que va a ser elegida
  if pronostico_b >= pronostico_a:
    ruta = "A"
  else:
    ruta = "B"
  
  #Generar ahora el tiempo real 
  if ruta == "B":
    tiempo_real = genUnif(45,75)
  else:
    if situacion_a == "sin congestion, pronostico preciso" or situacion_a == "sin congestion, pronostico impreciso":
      tiempo_real = genUnif(30,40)
    elif situacion_a == "congestion, pronostico preciso" or situacion_a == "congestion, pronostico impreciso":
      tiempo_real = genUnif(60,90)
  
  return tiempo_real

#Generamos un total de n observaciones de la variable de interes (duracion del viaje de Gloria)
n = 10000
obs = []
for i in range(n):
  obs.append(simulacion())

#Entonces el promedio de la duracion de la duracion del viaje y un intervalo de confianza para el mismo son:
datos = intConf(obs)
print(f"Duracion promedio del viaje: {datos[0]} \nintervalo de confianza: {datos[2]}")

#Numero de simulaciones para que la precision de un intervalo de confianza al 95% sea de un minuto
n = (1.96*datos[1])**2
print(f"El numero de simulaciones necesario para una precision de un minuto es de: {n}, redondeado por arriba son {math.ceil(n)}")

#Estimacion de la probabilidad de llegar tarde habiendo salido 40 minutos antes
puntualidad = []
for i in obs:
  if 40-i >= 0:
    puntualidad.append("temprano")
  else:
    puntualidad.append("tarde")

estimacion = puntualidad.count("tarde")/len(puntualidad)
precisionEst = 1.96*math.sqrt((estimacion*(1-estimacion))/len(puntualidad))
print(f"\nEstimacion P(llegar tarde dado que sali 40 minutos antes) = {estimacion} \nIntervalo de confianza al 95%: ({estimacion - precisionEst},{estimacion + precisionEst})")

Duracion promedio del viaje: 47.25659669270589 
intervalo de confianza: (46.930343999088215, 47.582849386323566)
El numero de simulaciones necesario para una precision de un minuto es de: 1064.4082009278982, redondeado por arriba son 1065

Estimacion P(llegar tarde dado que sali 40 minutos antes) = 0.4053 
Intervalo de confianza al 95%: (0.3956773798502902,0.41492262014970976)


#**Examen practico 2**

###Enunciado: https://drive.google.com/file/d/1upjqMd78tPcJrPxvKqIlooAlC4cobjW3/view?usp=sharing###

In [ ]:
import math
from random import random
import statistics
import pandas as pd

#Implementacion del metodo de boxmuller para generar variables aleatorias normales
def boxmuller(media, ds):

  # Genera dos números aleatorios uniformemente distribuidos en el intervalo (0, 1)
  u1 = random()
  u2 = random()

  # Aplica la transformación de Box-Muller
  z1 = math.sqrt(-2 * math.log(u1)) * math.cos(2 * math.pi * u2)
  #z2 = math.sqrt(-2 * math.log(u1)) * math.sin(2 * math.pi * u2)

  #Desestandarizacion
  x1 = ds*z1 + media
  #x2 = ds*z2 + media

  return x1

#Esta funcion calcula el intervalo de confianza al 95% de un conjunto de observaciones, devuelve una lista con varios elementos
def intConf(muestra):
  mediaMuestra = statistics.mean(muestra)
  desviacionMuestra = statistics.stdev(muestra)
  precision = 1.96*desviacionMuestra/math.sqrt(len(muestra))
  res = [mediaMuestra, desviacionMuestra, (mediaMuestra-precision, mediaMuestra+precision)]
  return res

#Calculo de los costos pasando la cantidad de habitaciones y los datos (generados con boxmuller)
def simulacion(cant, datos):
  costoHabitacion = 50
  costoHabitacionExtra = 80

  #Se quiere minimizar costos, por lo que la idea es probar diferentes cantidades de habitaciones reservadas

  cantidadHabitacionesReservadas = cant

  costos = []
  for i in datos:
    if i > cantidadHabitacionesReservadas:
      costos.append(cantidadHabitacionesReservadas*costoHabitacion + (i - cantidadHabitacionesReservadas)*costoHabitacionExtra)
    else:
      costos.append(cantidadHabitacionesReservadas*costoHabitacion)
  
  return costos

######################################################################
n = 10000

obs = [boxmuller(5000, 1000) for i in range(n)]

print(intConf(simulacion(5000, obs)))        #Un buen comienzo es la media

#Tomando como base este valor probamos valores cercanos para tratar de optimizar los costos

#Una sola simulacion

nroReservas = []
costoProm = []
desvEst = []
for i in range(3000, 7000, 100):
  datos = intConf(simulacion(i, obs))
  nroReservas.append(i)
  costoProm.append(datos[0])
  desvEst.append(datos[1])

print("\n",pd.DataFrame({"Reservas": nroReservas, "Costo promedio": costoProm, "Desviacion estandar": desvEst}),"\n")

#Muchas simulaciones

nroReservas = []
costoProm = []
desvEst = []
for i in range(3000, 7000, 100):
  promedioCostos = []
  for j in range(100):
    obs = [boxmuller(5000, 1000) for i in range(n)]
    costos = simulacion(i, obs)
    promedioCostos.append(statistics.mean(costos))
  datos = intConf(promedioCostos)
  nroReservas.append(i)
  costoProm.append(datos[0])
  desvEst.append(datos[1])

print("\n",pd.DataFrame({"Reservas": nroReservas, "Costo promedio": costoProm, "Desviacion estandar": desvEst}),"\n")

#Como se puede ver el optimo estara aproximadamente entre 4500 y 4700 habitaciones, acotemos mas este numero

[281680.1181364223, 46793.04422963658, (280762.97446952143, 282597.26180332317)]

     Reservas  Costo promedio  Desviacion estandar
0       3000   309864.180319         78716.560030
1       3100   307069.081049         78310.082290
2       3200   304334.222949         77807.160073
3       3300   301660.957738         77215.499874
4       3400   299068.119568         76512.293481
5       3500   296561.280500         75701.002597
6       3600   294153.931871         74773.540322
7       3700   291871.433220         73706.532505
8       3800   289735.029902         72487.324995
9       3900   287755.123120         71121.466054
10      4000   285945.275796         69611.826611
11      4100   284339.789208         67937.891436
12      4200   282948.960278         66111.821760
13      4300   281797.939727         64130.314097
14      4400   280914.415603         61989.833263
15      4500   280318.201230         59696.673208
16      4600   280002.822367         57282.788614
17      4700   27

In [ ]:
nroReservas = []
costoProm = []
desvEst = []

for i in range(4500, 4700, 10):
  promedioCostos = []
  for j in range(100):
    obs = [boxmuller(5000, 1000) for i in range(n)]
    costos = simulacion(i, obs)
    promedioCostos.append(statistics.mean(costos))
  datos = intConf(promedioCostos)
  nroReservas.append(i)
  costoProm.append(datos[0])
  desvEst.append(datos[1])

print("\n",pd.DataFrame({"Reservas": nroReservas, "Costo promedio": costoProm, "Desviacion estandar": desvEst}),"\n")

#Como se puede notar el optimo esta aproximadamente entre 4610 y 4630, acotemos una ultima vez para hallar lo que para nosotros seria el optimo


     Reservas  Costo promedio  Desviacion estandar
0       4500   280741.825880           551.671209
1       4510   280763.356772           632.299564
2       4520   280727.207955           567.091269
3       4530   280707.799519           563.570534
4       4540   280632.360658           633.852363
5       4550   280509.444913           590.279778
6       4560   280403.507656           577.911195
7       4570   280530.023155           536.196054
8       4580   280553.319403           522.103542
9       4590   280451.328619           582.950782
10      4600   280506.368727           581.280445
11      4610   280404.792476           533.880217
12      4620   280272.143554           553.468300
13      4630   280345.126222           622.798888
14      4640   280323.896987           540.359668
15      4650   280300.478413           608.342950
16      4660   280372.917832           576.492107
17      4670   280381.025145           568.132280
18      4680   280349.945305           524.42902

In [ ]:
nroReservas = []
costoProm = []
desvEst = []
intervalo = []
for i in range(4600, 4621):
  promedioCostos = []
  for j in range(100):
    obs = [boxmuller(5000, 1000) for i in range(n)]
    costos = simulacion(i, obs)
    promedioCostos.append(statistics.mean(costos))
  datos = intConf(promedioCostos)
  nroReservas.append(i)
  costoProm.append(datos[0])
  desvEst.append(datos[1])
  intervalo.append(datos[2])

print("\n",pd.DataFrame({"Reservas": nroReservas, "Costo promedio": costoProm, "Desviacion estandar": desvEst, "Intervalo": intervalo}),"\n")

#Basados en el promedio tomaremos como optimo reservar 4619 habitaciones (En la fila 19 se halla el intervalo de confianza)


     Reservas  Costo promedio  Desviacion estandar  \
0       4600   280518.882433           621.607296   
1       4601   280319.275726           633.005661   
2       4602   280398.567953           521.298955   
3       4603   280433.233273           576.891643   
4       4604   280467.393534           630.743876   
5       4605   280472.986081           556.440444   
6       4606   280403.677239           597.151333   
7       4607   280365.573878           577.044453   
8       4608   280406.149240           560.208381   
9       4609   280419.181234           548.595532   
10      4610   280352.242655           541.488969   
11      4611   280399.929727           550.910601   
12      4612   280401.280179           587.174812   
13      4613   280373.060405           595.664229   
14      4614   280374.456111           547.271029   
15      4615   280419.050413           571.179641   
16      4616   280353.305853           569.272758   
17      4617   280377.884569           545.4

In [13]:

#Calculemos el numero de simulaciones necesaria para estimar el costo con una precision de 1000 (Obtenemos una muestra en el optimo para la desviacion estandar)
obs = [boxmuller(5000, 1000) for i in range(10000)]
nuevaMuestra = simulacion(4619, obs)
n = (1.96*statistics.stdev(nuevaMuestra)/1000)**2

print(f"La cantidad de simulaciones necesaria para estimar el costo con una precision de 1000 es de {math.ceil(n)}")

La cantidad de simulaciones necesaria para estimar el costo con una precision de 1000 es de 12631
